In [3]:
import pandas as pd, numpy as np
from geopy.geocoders import Nominatim
final_data = []
geolocator = Nominatim(user_agent="Gmaps")
print("Enter source : ")
source=input()
loc = geolocator.geocode(source)
print(loc.address)
src_lat=loc.latitude
src_lng=loc.longitude
for i in range(1,6) :
    print("Enter destination ",i," : ")
    address=input()
    location = geolocator.geocode(address)
    print(location.address)
    lng=location.latitude
    lat=location.longitude
    print((location.latitude, location.longitude))
    data=[address,lng,lat]
    final_data.append(data)
labels = ['Place Name','Latitude', 'Longitude']
dataframe_1 = pd.DataFrame.from_records(final_data, columns=labels)

Enter source : 
Dr. Babasaheb Ambedkar International Airport
Dr. Babasaheb Ambedkar International Airport, Ring Road, Bandu Soni Layout, Dhantoli, Nagpur Urban Taluka, Nagpur District, Maharashtra, 440001, India
Enter destination  1  : 
Itwari
Itwari Junction, NH53, Gandhi Baug, Nagpur Urban Taluka, Nagpur District, Maharashtra, 440001, India
(21.1575034, 79.1188261)
Enter destination  2  : 
Empress Mall
Empress Mall, Shri Bejonji Mehta Road, Santra Bazaar, Gandhi Baug, Nagpur Urban Taluka, Nagpur District, Maharashtra, 440018, India
(21.1480866, 79.0932379)
Enter destination  3  : 
RBI Square
RBI Square, NH47;NH53, Santra Bazaar, Sadar Bazaar, Nagpur Urban Taluka, Nagpur District, Maharashtra, 440001, India
(21.1526445, 79.0809738)
Enter destination  4  : 
Yashwant Stadium
Yashwant Stadium, North Ambazari Road, Shaniwari, Dhantoli, Nagpur Urban Taluka, Nagpur District, Maharashtra, 440001, India
(21.139397250000002, 79.08226823640393)
Enter destination  5  : 
Katol Road
Katol Road, Ch

In [6]:
dataframe_1

,Place Name,Latitude,Longitude
0,Itwari,21.157503,79.118826
1,Empress Mall,21.148087,79.093238
2,RBI Square,21.152645,79.080974
3,Yashwant Stadium,21.139397,79.082268
4,Katol Road,21.170447,79.075094


In [4]:
import osmnx as ox
import networkx as nx
place="Nagpur,Maharashtra,India"
G=ox.graph_from_place(place,network_type='drive',simplify=True)
edges=ox.graph_to_gdfs(G,nodes=True,edges=True)

In [7]:
final_data1=[]
latlng=[]
for i in dataframe_1.itertuples():
    place=i[1]
    lat=i[2]
    lng=i[3]
    latlng.append((place,lat,lng))
    src= ox.get_nearest_node(G, (src_lat,src_lng))
    dest = ox.get_nearest_node(G, (lat,lng))
    route=nx.shortest_path_length(G,source=src,target=dest,weight='length')
    r=round(float(route/1000),2)
    data=[source,place,r]
    final_data1.append(data)
labels = ['Source','Destination', 'Distance(km)']
dataframe_2 = pd.DataFrame.from_records(final_data1, columns=labels)
dataframe_2.to_csv('project_map.csv')
dataframe_2

,Source,Destination,Distance(km)
0,Dr. Babasaheb Ambedkar International Airport,Itwari,11.70
1,Dr. Babasaheb Ambedkar International Airport,Empress Mall,8.53
2,Dr. Babasaheb Ambedkar International Airport,RBI Square,7.88
3,Dr. Babasaheb Ambedkar International Airport,Yashwant Stadium,6.76
4,Dr. Babasaheb Ambedkar International Airport,Katol Road,10.28


In [8]:
import folium
mapit = folium.Map( location=[src_lat,src_lng], zoom_start=12 )
folium.Marker(location=[ src_lat,src_lng ],popup=source, icon= folium.Icon(color='red',icon = 'plane') , radius=8 ).add_to(mapit)
#icon= folium.Icon(color='red',icon = 'info-sign')
for coord in latlng:
    folium.Marker( location=[ coord[1], coord[2] ],popup=coord[0], fill_color='#43d9de', radius=8 ).add_to( mapit )
mapit.save( 'project_map.html')
mapit